In [9]:
from konlpy.tag import Okt
import networkx as nx
import numpy as np
from itertools import combinations
from collections import Counter

In [53]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [26]:
okt = Okt()

In [27]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [74]:
def extract_keywords(text, num_keywords=10):
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    # 단어들의 출현 빈도수 계산
    word_counts = Counter(words)
    
    # 그래프 생성
    graph = nx.Graph()
    
    # 단어 노드 추가
    for word, count in word_counts.items():
        if count > 1:  # 최소 출현 빈도 조건
            graph.add_node(word, count=count)
    
    # 단어의 연결을 위한 윈도우 크기 설정
    window_size = 4
    
    # 윈도우 안에서 단어 간의 연결 설정
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        for w1, w2 in combinations(window_words, 2):
            if graph.has_node(w1) and graph.has_node(w2):
                if graph.has_edge(w1, w2):
                    graph[w1][w2]['weight'] += 1
                else:
                    graph.add_edge(w1, w2, weight=1)
    
    # PageRank 계산
    rank = nx.pagerank(graph, weight='weight')
    
    # 상위 num_keywords개의 키워드 추출
    top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]
    
    return [keyword for keyword, _ in top_keywords]

In [68]:
# 병렬 처리를 위한 함수
def parallel_keyword_extraction(docs, num_keywords=5):
    # 프로세스 수 설정 (CPU 코어 수 사용)
    num_processes = 4
    
    with Pool(num_processes) as pool:
        # 병렬로 키워드 추출하면서 진행 상황 표시
        results = []
        for result in tqdm(pool.imap(extract_keywords, [(doc, num_keywords) for doc in docs]), total=len(docs)):
            results.append(result)
    
    return results

In [92]:
def parse_text_file(file_path):
    result_dict = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 줄의 앞뒤 공백 제거
            line = line.strip()

            # C?? 형식의 코드와 나머지 문자열 분리
            if line:
                parts = line.split(' ', 1)  # 첫 번째 공백으로 나눔
                if len(parts) == 2:
                    key = parts[0]
                    values = parts[1].split(',')  # 쉼표와 공백으로 구분하여 리스트 생성
                    result_dict[key] = values

    return result_dict

In [100]:
def count_matches(keyword_list, reference_list):
    return sum(1 for keyword in keyword_list if keyword in reference_list)

In [29]:
text = '이 문서는 텍스트 마이닝을 테스트하는 샘플 문서입니다. 문서에서 중요한 키워드를 추출하는 방법을 알아보겠습니다. 키워드 추출은 자연어 처리에서 중요한 작업입니다.'

In [30]:
keywords = extract_keywords(text, num_keywords=5)

In [31]:
keywords

['워드', '추출', '문서']

In [32]:
# 예시 문장
documents = [
    '이낙연 측근이 말하는 ‘치밀한’ 사면론 카드 “결국 문재인 대통령이 어떤 판단을 하느냐에 달렸다. 대통령이 사면을 결정하면 지지층의 반발도 어느 정도 잦아들 가능성이 있다.” 이낙연 더불어민주당 대표가 새해 첫 메시지로 던진 이명박·박근혜 두 전직 대통령 사면 관련 논란에 대한 이 대표 측 한 인사의 말이다. 정치권에선 이 대표 사면론에 대해 비판 일색이었다. 여권 내부에선 ‘사과도 없는 범죄자들을 어떻게 사면하느냐’, ‘특히 박근혜씨는 형사범죄뿐 아니라 국민들의 판단(탄핵)도 있었다’ 등의 의견이 있고, 야권에서도 ‘이낙연 지지율 반등 카드’라는 지적과 함께 김종인 국민의힘 비상대책위원장의 두 전직 대통령 범죄 관련 사과로 환영만 할수 없다는 분위기도 있다. 해당 관계자는 미디어오늘에 “당장은 비난이 많을 수밖에 없고 나도 원론적 입장만 따지면 사면 반대입장이지만 이 대표 입장에선 정치적으로 고려한 결정 아니겠느냐”고 했다. 여기서 ‘정치적 고려’란 문 대통령이 느끼는 정치적 부담과 이 대표 대권 행보를 뜻한다. 문 대통령 입장에선 자신의 임기 직전 두 전직 대통령이 모두 옥에 갇혔고, 박근혜씨의 경우 전직 대통령 중 최장기간(약 3년10개월) 수감 중인데 이는 법적인 문제를 떠나 문 대통령에게 정치적 부담으로 느껴진 게 사실이라고 전했다. 또한 호남 인사로 분류되는 이 대표가 향후 대권행보에서 영남권 지지를 얻을 돌파구이기도 하다는 게 해당 관계자의 설명이다. 이런 가운데 6일 리얼미터·오마이뉴스 여론조사를 보면 두 전직 대통령 사면에 대해 찬성(47.7%)과 반대(48%) 의견은 팽팽했다. 정치권과 언론에서 사면론에 비판적 의견이 지배했던 것과 다른 분위기다. 민주당 지지층에선 반대(88.8%), 국민의힘 지지층에선 찬성(81.4%)이 압도적으로 나타난 것을 봐도 ‘표계산’의 관점에서 볼 때 사면론 카드를 실패했다고 단정하긴 어렵다. 또한 7일 조선일보는 “박근혜는 사면, 이명박은 보류 검토”란 기사에서 ‘여권 핵심 관계자’의 말을 인용해 박근혜씨 사면을 검토한다는 내용을 전했다. 이는 이 대표 쪽 관계자일 가능성이 높다. 조선일보 취재에 청와대 관계자는 “사면 관련 입장이 없다”고 했고, 해당 보도 이후 청와대 대변인은 “사면을 검토한 적 없다”고 했지만 여당 대표가 실제 사면을 건의할 경우 청와대가 이를 일방적으로 외면할 수 없는 것도 사실이다. 이 대표 쪽 관계자는 미디어오늘에 “문 대통령과 이 대표가 지난 연말에 만났는데 그 자리에서 무엇을 논의했는지는 당사자들만 알 수 있다”고 선을 그으면서도 “두분은 서로 어떤 생각을 하는지 알만한 각별한 사이”라며 사전교감 가능성을 열어뒀다. 이 대표는 사면건의와 영수회담 제안에 대해 “대통령과 사전교감이 없었다”고 말해왔다. 사전교감 여부를 떠나 결과적으로 공은 대통령에게 넘어간 분위기다. 문 대통령이 이 대표 건의를 받아들여 사면할 경우 이 대표는 그가 말한대로 ‘대통령에 대한 충정’을 보이면서 동시에 영남·보수성향 지지층에게도 어필하는 두 마리 토끼를 잡게 된다. 문 대통령이 최종적으로 임기 내에 사면을 단행하지 않더라도 이 대표에 대한 메시지를 내놓을 것이란 예상도 가능하다. 여권의 유력 대선주자인 이 대표가 혼자서 소위 ‘쇼’를 한 모양새가 되는 건 정권 입장에서도 부담스러운 일이기 때문이다. 이 대표는 사면 건의를 꺼내든 이유로 ‘국민통합’을 말했다. 이는 총리 시절부터 그가 구상해 온 생각인 만큼 앞으로 국민통합과 관련한 의제를 더 던질 것이라고 해당 관계자는 전했다. 이달 내 진행할 것으로 보이는 문 대통령과 이 대표의 신년기자회견에서 어떤 메시지가 나올지 주목된다.',
    '국민의힘 추미애-문재인까지 고발하자 민주 “황당무계” 국민의힘이 서울동부구치소의 코로나19 감염자 급증 및 사망자 발생 책임을 물어 추미애 법무부장관을 형사고발했다. 국민의힘은 또 윤석열 검찰총장을 불법적으로 징계했다며 추 장관과 함께 문재인 대통령까지 검찰에 고발했다. 이에 더불어민주당은 황당무계하다는 반응을 내놓았다. 이들은 또 기승전-검찰고발이냐며 모든 것을 검찰총장에게 해결해달라고 하느냐고 반문했다. 국민의힘 법제사법위원회 소속 의원들은 6일 오후 기자회견에서 추 장관을 고발한 사유에 대해 “법무부 산하 서울동부구치소의 코로나19 감염자에 대한 격리수용 등을 제대로 관리하지 않아 수용자 인권과 생명을 제대로 지키지 못하고 5일 현재 사망자 1명, 감염자 1085명에 이르게 한 혐의”라고 주장했다. 이들은 “지난해 11월27일 서울동부구치소 직원 1명이 확진 판정을 받은 이후 서울동부치소는 역학조사 및 접촉자 분리를 제대로 실시하지 않고 전수조사 대응이 늦었으며, 최초 확진자 발생 전 마스크도 지급하지 않았다”며 “지난해 9월엔 보건마스크를 자비로 구매하게 허가해 달라는 여주교도소 재소자의 진정을 기각하는 등 수감자들의 생명·신체의 위험을 방치하고 사망자와 감염자를 낳아 업무상과실·중과실치사상, 직무유기 혐의를 저질렀다”고 주장했다. 국민의힘은 “추 장관은 서울동부구치소의 확진자가 700명을 넘어선 지난 12월 29일, 첫 확진자가 나온 지 32일이 지나서야 서울동부구치소를 찾았다”며 “그 전날까지 추 장관은 ‘윤 총장을 탄핵시켜야 한다’는 여당 의원의 글을 SNS에 올리며 연일 윤 총장 찍어내기에 정신이 팔려 있었다”고 비판했다. 문 대통령과 추 장관의 윤석열 검찰총장 징계 건도 고발했다. 국민의힘은 “윤석열 검찰총장에 대한 감찰을 법무부 감찰규정 등 법령에 위반되게 진행했고, 재량권을 일탈하여 직무집행정지처분을 했고, 의사정족수도 못 갖춘 절차상 심각한 하자가 있음에도 추미애 장관과 문재인 대통령은 직권을 남용해 윤 총장 징계를 강행했다”며 “검찰의 철저한 수사를 당부 드린다”고 촉구했다. 이에 더불어민주당은 서울동부구치소 집단감염 비판을 받겠다면서도 고발까지 나선 것은 보복수사에 해당한다고 반발했다. 문 대통령까지 고발한 것 역시 황당무계하다는 반응이다. 김태년 더불어민주당 원내대표는 7일 오전 정책조정회의에서 추 장관 고발을 두고 “이성과 품격을 내던진 야당 고발 정치에 깊은 유감”이라고 밝혔다. 그는 “동부구치소 발 집단 감염 초기 대응은 비판할 수 있고, 추장관도 책임을 통감하며 거듭 사과했다”면서도 “하지만 추장관 고발은 검찰 끌여 들어 보복수사하겠다는 잔인한 정쟁”이라고 비판했다. 김 원내대표는 국민의힘의 추 장관 고발이 처음이 아니라며 울산시장 선거개입 의혹, 아들 병역 관련, 윤석열 징계 추진 등을 들어 “국민의힘이 추 장관에 묻지마 고발 되풀이하고 있다”고 했다. 그는 “국회에서 정치로 해결해야할 일들과 국정운영 사안까지 검찰 고발을 남발했다”며 “정치와 행정 영역 판단을 검찰과 법원에 넘기는 것은 민주주의를 후퇴시키는 행위”라고 비판했다. 김 원내대표는 특히 윤 총장 징계 재가를 빌미로 문 대통령 고발도 나선 것과 관련해서는 “황당무계하다”며 “정치의 금도를 지켜달라”고 했다. 이어 그는 “민주주의의 가드레일 부수는 묻지마 고발을 철회하기 바란다”며 “국민 갈등과 정치 불신을 키우는 무차별적 정쟁 유발에는 단호히 대응하겠다”고 강조했다. 강선우 더불어민주당 대변인도 논평을 내어 국민의힘의 고발 고질병이 또 도졌다며 ‘기승전 고발정치’에 몰두하고 있다고 밝혔다. 강 대변인은 “대통령의 인사권까지 고발하고 나섰다”며 “검찰총장을 앞세워 정치적 이득을 얻어 보려는 속셈인것인지, 아니면 스스로 일을 풀어나갈 힘이 없어 무능함을 고소 고발로 감추려는 것인지 알 수 없다”고 덧붙였다.',
    '윤호중 “인력 전혀 안줄여 놀라” 검찰 개혁 비협조 비판 검찰이 검경 수사권 조정이 됐는데도 기소권 중심의 조직으로 개편하거나 수사권 인력을 전혀 조정하지 않고 있다는 지적이 제기돼 주목된다. 특히 이것이 검찰이 수사권 조정을 흔쾌히 받아들이지 않고 있고, 검찰개혁에 협조하지 않으려는 것 아니냐는 목소리도 나왔다. 윤호중 국회 법제사법위원장은 7일 더불어민주당 당대표 회의실에서 열린 ‘더불어민주당 검찰개혁특위 3차회의’에서 이같이 전했다. 윤 위원장은 특위 출범 후 엊그제 2차회의에서 검경 수사권 조정후 검찰의 이행 상황 현황을 점검한 결과 “검찰개혁특위를 즉각 가동해 중단없는 2단계 검찰개혁에 착수한 일을 정말 잘했구나라고 확인했다”고 지적했다. 그 이유를 두고 윤 위원장은 “검찰이 직접 수사해오던 범위가 대폭 축소돼 6대 범죄 중에서도 중대 규모 범죄에 대해서만 수사하게 돼 있고 검찰의 직접 수사건수도 4분의 1 이하로 줄어들게 됐는데도 검찰은 그동안 운영해왔던 검찰 수사 인력을 전혀 조정하지 않고 있었다”며 “수사권이 줄었으니 기소권을 중심으로 운영하려면 기소권 중심으로 개편해야 하는데, 이 또한 전혀 이뤄지지 않고 있었다”고 전했다. 윤 위원장은 위원들이 다들 굉장히 놀랐다며 “검찰은 지금도 검찰 수사권 조정을 흔쾌히 받아들이고 있지 않은 상황이었다”고 평가했다. 윤 위원장은 “국회가 법 개정으로 검찰개혁을 추진해왔고 앞으로 해나갈텐데 검찰이 적극 협조하려는 의사 있는지 의심하지 않을 수 없다”며 “더욱더 검찰개혁에 매진해야겠다는 그런 사실을 확인했다”고 강조했다. 윤 위원장은 향후 검찰개혁 특위가 입법과제를 발굴해 오는 2월 초까지 입법안을 마련해 국회에 제출할 예정이라고 덧붙였다. 이낙연 더불어민주당 대표는 지난해 검경수사권 조정을 통해 6대 범죄만 검찰이 직접 수사하도록 하는 형사소송법 개정한 것을 들어 “개정 형사소송법이 1일부터 이미 시행되고 있다”고 전했다. 이 대표는 이와 함께 공수처 출범이 검찰의 기소독점에 예외를 만드는 70년 형사사법 역사가 새로운 변화를 맞는다고도 평가했다. 이어 이 대표는 검찰의 기소와 수사권 분리를 두고 “불공정한 수사와 선택적 기소를 포함한 검찰권 남용 방지를 포함해 검찰개혁의 추가 과제를 신속하고 과감하게 이행해 나가야 한다”며 “검찰청법과 형소법 개정안을 다음달 2월까지 국회 제출하고 그 안을 토대로 검찰개혁 추가 과제를 신속 착실하게 추진해가도록 하겠다”고 밝혔다. 이 대표는 “제도에 못지 않게 조직 문화나 구성원 의식 변화 수반돼야 한다”며 “할 수 있다면 그런 문제까지 포함해서 검찰이 문자 그대로 국민의 검찰 성숙한 민주검찰로 거듭나도록 해야 한다”고 촉구했다.',
]

In [34]:
extract_keywords(documents[0])

['사면', '대표', '대통령', '관계자', '박근혜', '국민', '입장', '정치', '관련', '전직']

In [43]:
doc_keywords = []

In [44]:
for doc in tqdm(documents) :
    keywords = extract_keywords(doc)
    doc_keywords.append(keywords)

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.15it/s]


In [45]:
doc_keywords

[['사면', '대표', '대통령', '관계자', '박근혜', '국민', '입장', '정치', '관련', '전직'],
 ['고발', '장관', '국민', '동부', '구치소', '감염', '대통령', '비판', '검찰총장', '검찰'],
 ['검찰', '개혁', '수사권', '조정', '위원장', '대표', '수사', '전혀', '특위', '기소권']]

In [37]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/combined_news.tsv', sep='\t')

In [38]:
len(df)

3611424

In [46]:
df.columns

Index(['id', 'topic', 'text'], dtype='object')

In [47]:
documents = df.text

In [48]:
len(documents)

3611424

In [78]:
df_p = df[df.topic == '정치']

In [84]:
len(df_p)

555588

In [83]:
del df

In [85]:
documents = df_p.text

In [86]:
doc_keywords = []

for doc in tqdm(documents) :
    keywords = extract_keywords(doc)
    doc_keywords.append(keywords)

100%|████████████████████████████████| 555588/555588 [41:10:21<00:00,  3.75it/s]


In [87]:
doc_keywords[:100]

[['대통령', '정부', '녹색', '개혁', '지원', '일자리', '라며', '제정부', '비상', '또한'],
 ['원내대표', '협상', '홍준표', '회담', '문국현', '국회', '인정', '교섭단체', '얼굴', '오늘'],
 ['앵커', '생각', '대해', '출신', '방송', '뉴스', '사실', '비판', '파업', '발언'],
 ['언론사', '일부', '앵커', '현재', '재벌', '김주하', '보도', '파업', '우리', '인터뷰'],
 ['의원', '블로그', '국회', '최문순', '민주당', '모습', '본회의', '경위', '사진', '상황'],
 ['비상', '경제', '대책', '회의', '상황', '대통령', '황실', '실장', '대변인', '청와대'],
 ['발행', '관계자', '소식지', '홍보', '구청', '별로', '문제', '추진', '선관위', '공직선거법'],
 ['국회', '보도', '기자', '민주당', '경찰', '사무', '투입', '상황', '의원', '충돌'],
 ['민주당', '국민', '대표', '본회의', '법안', '한나라당', '민생', '처리', '악법', '농성'],
 ['지난', '분야', '공약', '대해', '선거', '대통령', '언론', '폐기', '동안', '홍위병'],
 ['대표', '농성', '본회의', '민주당', '의원', '악법', '시작', '해제', '정세균', '저지'],
 ['비상', '경제', '변인', '부대', '목표', '전략', '대책', '대통령', '성과', '회의'],
 ['선관위', '발행', '구청', '소식지', '관계자', '소식', '검토', '문제', '공직선거법', '대전'],
 ['지방선거', '선진당', '내년', '생각', '선거', '위원장', '대표', '가지', '의원', '한해'],
 ['교섭단체', '대표', '유지', '선진당', '독자', '창조', '선진', '모임', '창조한국당', '문국현'],
 [

In [89]:
df_p['keywords'] = doc_keywords

In [99]:
df_p.iloc[100].keywords

['정부', '의원', '정치', '민주당', '위해', '질문', '입법', '국정원', '법안', '정보']

In [91]:
df_p.to_csv(directory_path+'/combined_news_P.tsv', sep='\t', index=False)

In [93]:
file_path = '/home/osung/Downloads/KSIC_keyword.txt'
KSIC_key = parse_text_file(file_path)

In [95]:
KSIC_key

{'c101': ['도축장',
  '도축',
  '축산물',
  '도축업',
  '도축',
  '육류',
  '한우',
  '냉동',
  '토종닭',
  '축산',
  '마리',
  '사육',
  '고래고기',
  '햄',
  '소시지',
  '선지',
  '소고기',
  '돼지고기',
  '양고기',
  '오리고기',
  '닭고기',
  '훈제',
  '닭가슴살',
  '닭발',
  '닭갈비',
  '닭꼬치',
  '양꼬치',
  '돈까스',
  '돈가스',
  '족발',
  '갈비',
  '베이컨',
  '편육'],
 'c102': ['수산물',
  '양식',
  '수산',
  '광어',
  '항만',
  '해조',
  '어란',
  '어육',
  '캐비어',
  '어묵',
  '젓갈',
  '생선통조림',
  '생선페이스트',
  '건어물',
  '오징어',
  '우럭',
  '도미',
  '도다리',
  '가자미',
  '갈치',
  '고등어',
  '멍게',
  '해삼',
  '굴',
  '석화',
  '조개',
  '가리비',
  '키조개',
  '참돔',
  '돌돔',
  '줄돔',
  '방어',
  '쥐포',
  '먹태',
  '황태',
  '게장',
  '맛살',
  '생선가스',
  '생선까스',
  '액젓',
  '북어채',
  '황태채',
  '미역',
  '김부각',
  '구운김',
  '조미김'],
 'c103': ['농업',
  '농가',
  '농산물',
  '농협',
  '식품',
  '양파',
  '마늘',
  '과일',
  '채소주스',
  '과일주스',
  '과채주스',
  '알로에',
  '과실주스',
  '딸기잼',
  '사과잼',
  '포도잼',
  '과일페이스트',
  '과일통조림',
  '과일샐러드',
  '김치',
  '단무지',
  '건채소',
  '건과일',
  '장아찌',
  '무쌈',
  '쌈무',
  '오이지',
  '피클',
  '감귤주스',
  '녹즙',
  '매실액',
  '배즙',
  '사과즙',
 

In [96]:
combined_list = []

In [97]:
for key in KSIC_key :
    combined_list.extend(KSIC_key[key])

In [98]:
combined_list

['도축장',
 '도축',
 '축산물',
 '도축업',
 '도축',
 '육류',
 '한우',
 '냉동',
 '토종닭',
 '축산',
 '마리',
 '사육',
 '고래고기',
 '햄',
 '소시지',
 '선지',
 '소고기',
 '돼지고기',
 '양고기',
 '오리고기',
 '닭고기',
 '훈제',
 '닭가슴살',
 '닭발',
 '닭갈비',
 '닭꼬치',
 '양꼬치',
 '돈까스',
 '돈가스',
 '족발',
 '갈비',
 '베이컨',
 '편육',
 '수산물',
 '양식',
 '수산',
 '광어',
 '항만',
 '해조',
 '어란',
 '어육',
 '캐비어',
 '어묵',
 '젓갈',
 '생선통조림',
 '생선페이스트',
 '건어물',
 '오징어',
 '우럭',
 '도미',
 '도다리',
 '가자미',
 '갈치',
 '고등어',
 '멍게',
 '해삼',
 '굴',
 '석화',
 '조개',
 '가리비',
 '키조개',
 '참돔',
 '돌돔',
 '줄돔',
 '방어',
 '쥐포',
 '먹태',
 '황태',
 '게장',
 '맛살',
 '생선가스',
 '생선까스',
 '액젓',
 '북어채',
 '황태채',
 '미역',
 '김부각',
 '구운김',
 '조미김',
 '농업',
 '농가',
 '농산물',
 '농협',
 '식품',
 '양파',
 '마늘',
 '과일',
 '채소주스',
 '과일주스',
 '과채주스',
 '알로에',
 '과실주스',
 '딸기잼',
 '사과잼',
 '포도잼',
 '과일페이스트',
 '과일통조림',
 '과일샐러드',
 '김치',
 '단무지',
 '건채소',
 '건과일',
 '장아찌',
 '무쌈',
 '쌈무',
 '오이지',
 '피클',
 '감귤주스',
 '녹즙',
 '매실액',
 '배즙',
 '사과즙',
 '감말랭이',
 '고구마말랭이',
 '팜유',
 '중유',
 '정제유',
 '음식물',
 '두유',
 '옥수수기름',
 '피마자유',
 '참기름',
 '들기름',
 '마가린',
 '쇼트닝',
 '올리브유',
 '콩기름',
 '낙농',
 '낙농업',

In [101]:
# 각 행에 대해 일치하는 문자열의 개수를 세고 새로운 열에 저장
df_p['match_count'] = df_p['keywords'].apply(lambda x: count_matches(x, combined_list))

In [102]:
df_p.iloc[:100]

,id,topic,text,keywords,match_count
17,NIRW1900000001.37,정치,"이명박 대통령, '비상경제정부' 선언 이명박 대통령이 '비상경제정부'를 선언하고 나...","[대통령, 정부, 녹색, 개혁, 지원, 일자리, 라며, 제정부, 비상, 또한]",0
31,NIRW1900000001.58,정치,홍준표는 왜 회담장을 박차고 나왔나 국회 대치정국을 풀기 위한 여야의 최후 담판 과...,"[원내대표, 협상, 홍준표, 회담, 문국현, 국회, 인정, 교섭단체, 얼굴, 오늘]",0
32,NIRW1900000001.60,정치,"""저널리즘 차원에서 중요한 사건틀리다고 생각했다면 말도 안 했다"" MBC <뉴스데스...","[앵커, 생각, 대해, 출신, 방송, 뉴스, 사실, 비판, 파업, 발언]",0
46,NIRW1900000001.103,정치,"""MBC가 일부 재벌과 언론사 소유가 된다면..."" <뉴스데스크> 박혜진 아나운서,...","[언론사, 일부, 앵커, 현재, 재벌, 김주하, 보도, 파업, 우리, 인터뷰]",0
55,NIRW1900000001.120,정치,문순씨네 블로그? 국회의원도 현장중계 국회의원도 이젠 블로거? 최문순 의원이 만든 ...,"[의원, 블로그, 국회, 최문순, 민주당, 모습, 본회의, 경위, 사진, 상황]",1
...,...,...,...,...,...
1206,NIRW1900000001.2118,정치,"머쓱해진 홍준표... 100분토론서 ""정치공세 말라"" MBC <100분 토론>에 출...","[이메일, 원내대표, 청와대, 문건, 공세, 토론, 의장, 관련, 대책, 뉴스]",0
1209,NIRW1900000001.2121,정치,의정비 올렸지만 나아지지 않고 있는 의정활동 서울 강남구의회 의원들의 의정비가 20...,"[활동, 의원, 의정, 위원회, 상임, 정비, 인터넷, 인상, 방송, 주민]",1
1217,NIRW1900000001.2133,정치,"지역민 78% ""울산 북구 보궐선거 진보진영 단일화 해야"" 울산 북구 윤두환 국회의...","[후보, 주민, 지역, 민주, 진보, 일화, 울산, 진영, 조합원, 네트워크]",0
1222,NIRW1900000001.2139,정치,"""재정압박 빌미로 OBS에 MB특보 낙하산"" 경인지역새방송창사준비위원회(이하 창준위...","[방송, 사장, 경인, 선임, 특보, 차용, 민영, 출신, 지역, 경영]",0


In [104]:
df_p.match_count.min()

0

In [105]:
df = df_p.sort_values(by='match_count', ascending=False)

In [107]:
df[:100]

,id,topic,text,keywords,match_count
3578053,NWRW2200000004.5583,정치,이마트 과일 용기 ‘재생 플라스틱’으로 교체 이마트가 6월부터 과일·채소를 포장하는...,"[플라스틱, 용기, 재생, 포장, 라벨, 과일, 이마트, 스티커, 채소, 기존]",7
3577982,NWRW2200000004.5455,정치,"전기 비행기는 배터리가 너무 무거워… ‘수소 비행기’가 뜬다 ‘제트여객기 285g,...","[수소, 비행기, 전기, 배터리, 연료, 전지, 기반, 모터, 항공기, 리튬]",7
603614,NWRW1900000016.22077,정치,동북아 오일허브 사업 5170억 들여 ‘빈 깡통’ 정부가 세계 4대 오일 허브로 키...,"[배럴, 오일, 저장, 허브, 유제품, 탱크, 사업, 석유, 원유, 해당]",7
2772530,NWRW1900000008.25815,정치,"CJ 손경식 ＂조원동(前 청와대 경제수석), 대통령 뜻이라며 이미경 물러나라 했다＂...","[회장, 최씨, 롯데, 의원, 스포츠, 재단, 광고, 회사, 현대차, 정유]",5
3568084,NWRW2200000003.19451,정치,경제외교 나선 삼성·현대차·SK·LG… 한미정상회담서 잭팟 노린다 삼성·현대자동차·...,"[미국, 투자, 생산, 전기차, 백신, 현대차, 공장, 삼성, 배터리, 반도체]",5
...,...,...,...,...,...
482139,NLRW2000000006.5751,정치,"[경인정가]김한정, '택배 차량에도 유가보조금 지급' 개정 법안 대표발의 김한정(남...","[택배, 유가, 보조금, 차량, 지급, 대해, 개정안, 화물차, 대표, 발의]",4
3589608,NWRW2200000005.8678,정치,"반도체·전기차…미국에 44조원 푸는 4대 기업, 구체 내용은? 한·미 정상회담 기간...","[투자, 공장, 미국, 배터리, 계획, 전자, 반도체, 에스케이, 파운드리, 생산]",4
3575410,NWRW2200000004.993,정치,태양광·수소에 2조8000억(2021년부터 5년간) 투자… ‘쎄트렉아이(국내 우주 ...,"[한화, 금융, 기업, 위성, 사업, 우주, 투자, 미국, 에너지, 탱크]",4
657431,NLRW1900000040.4780,정치,"중형위성 조립공장, 진주가 적격 지난 2일 오후 진주시청 시민홀에서 '차세대 중형위...","[조립, 위성, 공장, 우주, 중형, 진주, 부품, 시험, 산업, 공청회]",4


In [108]:
df_p = df[df['match_count'] >= 1]

In [109]:
len(df_p)

62750

In [110]:
len(df)

555588

In [111]:
df_p.to_csv(directory_path+'/news_P_keyword.tsv', sep='\t', index=False)

In [112]:
p_keywords = [
    "민주", "선거", "의회", "헌법", "입법",
    "정당", "후보자", "정책", "캠페인", "유권자",
    "정부", "대통령", "총리", "내각", "상원의원",
    "국회의원", "외교", "국민투표", "로비", "하원의원", 
    "장관", "총리", "정치", "주의", "민주당", "국민의힘", 
    "더불어민주당", "공화당", "자민당", "조국혁신당", "녹색당", 
    "노동당", "국민당", "자유당", "진보당", "국회", "정치적", 
    "이념", "참정권"
]

In [115]:
p_keywords

['민주',
 '선거',
 '의회',
 '헌법',
 '입법',
 '정당',
 '후보자',
 '정책',
 '캠페인',
 '유권자',
 '정부',
 '대통령',
 '총리',
 '내각',
 '상원의원',
 '국회의원',
 '외교',
 '국민투표',
 '로비',
 '하원의원',
 '장관',
 '총리',
 '정치',
 '주의',
 '민주당',
 '국민의힘',
 '더불어민주당',
 '공화당',
 '자민당',
 '조국혁신당',
 '녹색당',
 '노동당',
 '국민당',
 '자유당',
 '진보당',
 '국회',
 '정치적',
 '이념',
 '참정권']

In [120]:
# 각 행에 대해 일치하는 문자열의 개수를 세고 새로운 열에 저장
df['p_match'] = df['keywords'].apply(lambda x: count_matches(x, p_keywords))

In [117]:
df_p[:10]

,id,topic,text,keywords,match_count,p_match
3578053,NWRW2200000004.5583,정치,이마트 과일 용기 ‘재생 플라스틱’으로 교체 이마트가 6월부터 과일·채소를 포장하는...,"[플라스틱, 용기, 재생, 포장, 라벨, 과일, 이마트, 스티커, 채소, 기존]",7,0
3577982,NWRW2200000004.5455,정치,"전기 비행기는 배터리가 너무 무거워… ‘수소 비행기’가 뜬다 ‘제트여객기 285g,...","[수소, 비행기, 전기, 배터리, 연료, 전지, 기반, 모터, 항공기, 리튬]",7,0
603614,NWRW1900000016.22077,정치,동북아 오일허브 사업 5170억 들여 ‘빈 깡통’ 정부가 세계 4대 오일 허브로 키...,"[배럴, 오일, 저장, 허브, 유제품, 탱크, 사업, 석유, 원유, 해당]",7,0
2772530,NWRW1900000008.25815,정치,"CJ 손경식 ＂조원동(前 청와대 경제수석), 대통령 뜻이라며 이미경 물러나라 했다＂...","[회장, 최씨, 롯데, 의원, 스포츠, 재단, 광고, 회사, 현대차, 정유]",5,0
3568084,NWRW2200000003.19451,정치,경제외교 나선 삼성·현대차·SK·LG… 한미정상회담서 잭팟 노린다 삼성·현대자동차·...,"[미국, 투자, 생산, 전기차, 백신, 현대차, 공장, 삼성, 배터리, 반도체]",5,0
2108048,NPRW2100000002.28127,정치,"공영운 현대차 사장 ""수소연료전지 수명 2배 늘리고, 가격은 반으로 줄인다"" 공영운...","[국회, 사장, 연료, 포럼, 전지, 가격, 수소, 자동차, 수명, 현대차]",5,1
3577054,NWRW2200000004.3805,정치,"年6%씩 커가는 CMO(의약품 위탁생산) 산업, 대기업·제약사 줄줄이 뛰어들어 SK...","[바이오, 코로나, 생산, 백신, 사업, 시장, 치료, 유전자, 위탁, 의약품]",5,0
1414420,NLRW2200000013.6153,정치,민주당 안호영 의원 경제 세일즈맨을 나섰다 더불어민주당 안호영 의원(완주·진안·무주...,"[자동차, 포스코, 공장, 사업, 현대, 의원, 상용차, 모빌리티, 트럭, 현대차]",5,0
2097994,NPRW2100000002.15188,정치,"비닐장갑 위 도장 '쾅', 마스크 쓰고 '찰칵'…달라진 '투표 인증샷' 제 21대 ...","[투표, 장갑, 비닐, 인증샷, 도장, 마스크, 사진, 투표소, 촬영, 코로나]",5,0
3575868,NWRW2200000004.1782,정치,화웨이·LG 빈자리를 차지하라 ‘가성비폰’ 전쟁 나선 삼성과 애플 삼성전자와 애플이...,"[애플, 전자, 가폰, 스마트폰, 갤럭시, 시장, 시리즈, 화웨이, 프리미엄, 아이폰]",5,0


In [118]:
len(df)

555588

In [121]:
df[:10]

,id,topic,text,keywords,match_count,p_match
3578053,NWRW2200000004.5583,정치,이마트 과일 용기 ‘재생 플라스틱’으로 교체 이마트가 6월부터 과일·채소를 포장하는...,"[플라스틱, 용기, 재생, 포장, 라벨, 과일, 이마트, 스티커, 채소, 기존]",7,0
3577982,NWRW2200000004.5455,정치,"전기 비행기는 배터리가 너무 무거워… ‘수소 비행기’가 뜬다 ‘제트여객기 285g,...","[수소, 비행기, 전기, 배터리, 연료, 전지, 기반, 모터, 항공기, 리튬]",7,0
603614,NWRW1900000016.22077,정치,동북아 오일허브 사업 5170억 들여 ‘빈 깡통’ 정부가 세계 4대 오일 허브로 키...,"[배럴, 오일, 저장, 허브, 유제품, 탱크, 사업, 석유, 원유, 해당]",7,0
2772530,NWRW1900000008.25815,정치,"CJ 손경식 ＂조원동(前 청와대 경제수석), 대통령 뜻이라며 이미경 물러나라 했다＂...","[회장, 최씨, 롯데, 의원, 스포츠, 재단, 광고, 회사, 현대차, 정유]",5,0
3568084,NWRW2200000003.19451,정치,경제외교 나선 삼성·현대차·SK·LG… 한미정상회담서 잭팟 노린다 삼성·현대자동차·...,"[미국, 투자, 생산, 전기차, 백신, 현대차, 공장, 삼성, 배터리, 반도체]",5,0
2108048,NPRW2100000002.28127,정치,"공영운 현대차 사장 ""수소연료전지 수명 2배 늘리고, 가격은 반으로 줄인다"" 공영운...","[국회, 사장, 연료, 포럼, 전지, 가격, 수소, 자동차, 수명, 현대차]",5,1
3577054,NWRW2200000004.3805,정치,"年6%씩 커가는 CMO(의약품 위탁생산) 산업, 대기업·제약사 줄줄이 뛰어들어 SK...","[바이오, 코로나, 생산, 백신, 사업, 시장, 치료, 유전자, 위탁, 의약품]",5,0
1414420,NLRW2200000013.6153,정치,민주당 안호영 의원 경제 세일즈맨을 나섰다 더불어민주당 안호영 의원(완주·진안·무주...,"[자동차, 포스코, 공장, 사업, 현대, 의원, 상용차, 모빌리티, 트럭, 현대차]",5,0
2097994,NPRW2100000002.15188,정치,"비닐장갑 위 도장 '쾅', 마스크 쓰고 '찰칵'…달라진 '투표 인증샷' 제 21대 ...","[투표, 장갑, 비닐, 인증샷, 도장, 마스크, 사진, 투표소, 촬영, 코로나]",5,0
3575868,NWRW2200000004.1782,정치,화웨이·LG 빈자리를 차지하라 ‘가성비폰’ 전쟁 나선 삼성과 애플 삼성전자와 애플이...,"[애플, 전자, 가폰, 스마트폰, 갤럭시, 시장, 시리즈, 화웨이, 프리미엄, 아이폰]",5,0


In [122]:
df.iloc[3].text

'CJ 손경식 ＂조원동(前 청와대 경제수석), 대통령 뜻이라며 이미경 물러나라 했다＂ \'최순실 국조\' 특혜성 집중 추궁 6일 열린 국회 국정조사특별위원회 청문회에서 여야(與野) 의원들은 대기업 총수들을 상대로 미르·K스포츠 재단 출연금 외에도 각 기업이 최순실(60·구속)씨 등에게 특혜성 지원을 했는지를 추궁했다. ◇최씨 광고회사에 일감 70억 준 현대차 국조특위에선 현대차가 최씨 지인의 회사인 KD코퍼레이션에 10억원대 제품 납품을 하도록 해준 일과 최씨의 광고회사인 플레이그라운드에 70억원의 광고를 발주한 일이 도마에 올랐다. 최씨와 안종범 전 청와대 정책조정수석 등에 대한 검찰 공소장에 따르면 안 전 수석으로부터 박근혜 대통령의 지시를 전달받은 현대차는 작년 2월 최씨의 딸 정유라씨의 초등학교 친구 부친이 운영하는 자동차 흡착제 제조회사 KD코퍼레이션과 10억5000만원 규모의 납품 계약을 체결했다. 검찰은 현대차가 올 4~5월엔 최씨의 광고회사 플레이그라운드에 70억원의 광고를 몰아줬다고도 공소장에서 밝혔다. 정몽구 현대차 회장은 "(현대차는) 세계적인 자동차 그룹인데 KD코퍼레이션한테 돈을 뜯긴 게 창피하지 않으신가"라는 더불어민주당 박범계 의원의 질의에 대해 "어떤 사정이 있는지는 모르겠지만 처음 듣는 얘기"라고 말했다. 정 회장은 또 "플레이그라운드에 광고를 주라는 요청을 받은 사실이 있느냐"는 새누리당 이만희 의원의 질의엔 "내가 직접 관련도 없을 뿐 아니라 기억도 나지 않는다"고 했다. 그러나 정 회장의 변호인은 "(2014년 말) 박 대통령과 정 회장의 독대 말미에 안 전 수석이 KD코퍼레이션을 도와주라는 얘기를 했다고 독대 자리에 있었던 김용환 부회장으로부터 전해들었다"고 말했다. ◇K스포츠재단의 롯데 70억, SK 80억 투자 요구 검찰 수사에선 K스포츠재단 측이 올 2~3월 박 대통령의 총수 독대 이후 롯데와 SK에 각각 70억원과 80억원을 요구한 사실도 드러났다. 공소장에 따르면 최순실씨는 지난 3월 "롯데와 다 얘기가 됐으니 돈